In [3]:
%%spark

In [4]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._

import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._


In [6]:

    val spark = SparkSession
      .builder
      .appName("TP Spark : Preprocessor")
      .getOrCreate()
    import spark.implicits._

org.apache.spark.sql.SparkSession$implicits$@56752c4c

In [13]:
val df: DataFrame = spark
      .read
      .option("header", true) // utilise la première ligne du (des) fichier(s) comme header
      .option("inferSchema", "true") // pour inférer le type de chaque colonne (Int, String, etc.)
      .csv("/Users/jeremieperes/MS Big data Télécom/P1/INF729 - Hadoop et Spark/Spark/spark_project_kickstarter_2019_2020/data/train_clean.csv")


In [14]:
 val dfCasted: DataFrame = df
      .withColumn("goal", $"goal".cast("Int"))
      .withColumn("backers_count", $"backers_count".cast("Int"))
      .withColumn("final_status", $"final_status".cast("Int"))

In [25]:
 def cleanCountry(country: String, currency: String): String = {
      if (country == null)
        currency
      if (country.length!=2)
        null
      else
        country
    }

    def cleanCurrency(currency:String): String = {
      if (currency.length != 3)
        null
      else
        currency
    }

cleanCountry: (country: String, currency: String)String
cleanCurrency: (currency: String)String


In [26]:
val df2: DataFrame = dfCasted.drop("disable_communication")

    val dfNoFutur : DataFrame = df2.drop("backers_count","state_changed_at")

    val cleanCountryUdf = udf(cleanCountry _)
    val cleanCurrencyUdf = udf(cleanCurrency _)

    val dfCountryCurrency = dfNoFutur
      .withColumn("country2",cleanCountryUdf($"country",$"currency"))
      .withColumn("currency2",cleanCurrencyUdf($"currency"))
      .drop("country","currency")

In [16]:
 val dfCleaned = dfCountryCurrency.filter($"final_status"===0 || $"final_status"===1)
      .withColumn("days_campaign",datediff(from_unixtime($"deadline"),from_unixtime($"launched_at")))
      .withColumn("launched_at",$"launched_at".cast("Float"))
      .withColumn("created_at",$"created_at".cast("Float"))
      .withColumn("hours_prepa",round(($"launched_at"-$"created_at")/3600,3))
      .drop("launched_at","deadline","created_at")
      .withColumn("name",lower($"name"))
      .withColumn("desc",lower($"desc"))
      .withColumn("keywords",lower($"keywords"))
      .withColumn("text",concat_ws(" ",$"name",$"desc",$"keywords"))
      .na.fill(Map("days_campaign" -> -1, "hours_prepa" -> -1,"goal" -> -1,"country2" -> "unknown","currency2" -> "unknown"))

In [29]:
dfCountryCurrency.display(10)

org.apache.spark.SparkException:  Task not serializable